# Exploring the ACLED API

The dataMisc function contains code to search the ACLED API

In [ ]:
import sys, os, requests, json
import pycountry

import pandas as pd
import geopandas as gpd

sys.path.insert(0, "../src")

import GOSTrocks.dataMisc as dMisc

%load_ext autoreload
%autoreload 2

In [ ]:
creds = dMisc.get_acled_creds()
email = creds['email']
api_key = creds['api_key']

iso3 = "UKR"
country_coding = pycountry.countries.get(alpha_3=iso3)
iso_code = country_coding.numeric
start_date = "2024-01-01"


In [ ]:
# Fetch all ACLED events since 2024-01-01 in selected country
acled_data = dMisc.acled_search(api_key, email, iso3=iso_code, start_date=start_date)
acled_data.groupby(['year', 'event_type']).count()['event_date']

In [ ]:
# Fetch adm1 boundaries for selected ISO3
# adm0 = dMisc.get_geoboundaries(iso3, "ADM0")
adm1 = dMisc.get_geoboundaries(iso3, "ADM1")

In [ ]:
# Query all events that are within the capital
query_geom = adm1.loc[adm1['shapeName'] == "Nairobi"].union_all()
acled_data = dMisc.acled_search(api_key, email, bounding_box=query_geom.bounds)
acled_data.groupby(['year', 'event_type']).count()['event_date']

In [ ]:
# Query all events that are within the capital with explicit field selection
query_geom = adm1.loc[adm1['shapeName'] == "Nairobi"].union_all()
### Defining the fields to be returned shortens the response time from 10.6 seconds to 6.2 seconds
cled_data = dMisc.acled_search(api_key, email, bounding_box=query_geom.bounds, fields=['year','event_type', 'geo_precision'])
acled_data.groupby(['year', 'event_type']).count()

In [ ]:
acled_data